关于感受野（receptive field）的概念和计算，请参考[该博客](https://www.cnblogs.com/ziytong/p/10697129.html)，或自行搜索该关键字。

# import

In [5]:
import matplotlib.pyplot as plt

In [2]:
from fastai.vision import models

In [3]:
from torch import nn

In [7]:
from ..exp import nb_anchors_loss_metrics
from ..exp import nb_resnet_ssd

# config

# compute receptive field

## functions

### featureMap_geometry

**函数名称 featureMap_geometry 的说明：**  
该函数的作用是分析各层的特征图的尺寸、感受野（receptive field)、jump，以及特征点起始中心（start），我们认为这些信息都是与几何相关的，所以用该名称。

In [4]:
def featureMap_geometry(m:nn.Module, size_in:int, subBranchs:list=[]):
    '''
    参考：https://www.cnblogs.com/ziytong/p/10697129.html
    --------------------
    参数：
    --m：神经网络模型，一个nn.Module对象
    --size_in：输入图片尺寸，假定输入图片是方形的，所以只需（只能）指定一个维度上的尺寸
    --subBranchs：分支模块的名称关键字。该函数只分析主干模块，不分析分支模块（例如resnet的残差连接），通过subBranchs指出模型定义中分支模块名称
    包含的关键字，函数内部通过这些关键字来判断一个模块是否分支模块。
    --------------------
    输出：
    --无返回值
    --打印各主干模块名称和对应特征图的尺寸(n)、感受野（receptive field）、jump、以及边缘特征像素的中心点（start)
    '''
    def _is_main(mn):
        "内部函数，通过检查模块名字是否包含subBranchs中的关键字来判断其是否分支模块"
        is_sub = False
        for sub in subBranchs:
            is_sub = is_sub or (sub in mn)
        return not is_sub
    
    ns = [size_in] # feature size
    rs = [1] # receptive field
    js = [1] # jump
    starts = [0.5] # start center
    mns = ['input'] # module names
    
    for mn,mm in m.named_modules():
        if _is_main(mn) and (isinstance(mm,nn.Conv2d) or isinstance(mm,nn.MaxPool2d)):
            mns += [mn]
            
            k = mm.kernel_size; k = k if isinstance(k,int) else k[0]
            p = mm.padding; p = p if isinstance(p,int) else p[0]
            s = mm.stride; s = s if isinstance(s,int) else s[0]
            
            ns += [int((ns[-1]+2*p-k)/s) + 1]
            rs += [rs[-1] + (k-1)*js[-1]]
            js += [js[-1] * s]
            starts += [starts[-1] + ((k-1)/2-p)*js[-1]]
            
    for mn,n,r,j,start in zip(mns,ns,rs,js,starts):
        print(f'{mn}: size={n},receptive={r},jump={j},start={start}')
            

## process

In [14]:
# 实例化你的模型
m = models.resnet34()

In [8]:
# m = nb_resnet_ssd.get_resnet34_1ssd()

In [15]:
# 观察模型内模块名称，找出分支模块名称的关键字
for mn,mm in m.named_modules():
    # 因为对特征图感受野等几何特征产生影响的只有conv和pool，我们只关心这两类模块
    if isinstance(mm,nn.Conv2d) or isinstance(mm,nn.MaxPool2d) or isinstance(mm,nn.AvgPool2d):
        print(mn)

conv1
maxpool
layer1.0.conv1
layer1.0.conv2
layer1.1.conv1
layer1.1.conv2
layer1.2.conv1
layer1.2.conv2
layer2.0.conv1
layer2.0.conv2
layer2.0.downsample.0
layer2.1.conv1
layer2.1.conv2
layer2.2.conv1
layer2.2.conv2
layer2.3.conv1
layer2.3.conv2
layer3.0.conv1
layer3.0.conv2
layer3.0.downsample.0
layer3.1.conv1
layer3.1.conv2
layer3.2.conv1
layer3.2.conv2
layer3.3.conv1
layer3.3.conv2
layer3.4.conv1
layer3.4.conv2
layer3.5.conv1
layer3.5.conv2
layer4.0.conv1
layer4.0.conv2
layer4.0.downsample.0
layer4.1.conv1
layer4.1.conv2
layer4.2.conv1
layer4.2.conv2


In [16]:
# 根据前一单元的输出来指定分支模块名称中的关键字
subBranchs = ['downsample']

In [17]:
# 指定输入图片尺寸
size_in = 776

In [18]:
# 打印各“主干模块”的特征图的几何信息
featureMap_geometry(m,size_in=size_in,subBranchs=subBranchs)

input: size=776,receptive=1,jump=1,start=0.5
conv1: size=388,receptive=7,jump=2,start=0.5
maxpool: size=194,receptive=11,jump=4,start=0.5
layer1.0.conv1: size=194,receptive=19,jump=4,start=0.5
layer1.0.conv2: size=194,receptive=27,jump=4,start=0.5
layer1.1.conv1: size=194,receptive=35,jump=4,start=0.5
layer1.1.conv2: size=194,receptive=43,jump=4,start=0.5
layer1.2.conv1: size=194,receptive=51,jump=4,start=0.5
layer1.2.conv2: size=194,receptive=59,jump=4,start=0.5
layer2.0.conv1: size=97,receptive=67,jump=8,start=0.5
layer2.0.conv2: size=97,receptive=83,jump=8,start=0.5
layer2.1.conv1: size=97,receptive=99,jump=8,start=0.5
layer2.1.conv2: size=97,receptive=115,jump=8,start=0.5
layer2.2.conv1: size=97,receptive=131,jump=8,start=0.5
layer2.2.conv2: size=97,receptive=147,jump=8,start=0.5
layer2.3.conv1: size=97,receptive=163,jump=8,start=0.5
layer2.3.conv2: size=97,receptive=179,jump=8,start=0.5
layer3.0.conv1: size=49,receptive=195,jump=16,start=0.5
layer3.0.conv2: size=49,receptive=227,j